<a href="https://colab.research.google.com/github/nrohrbach/KeyFiguresElectricalInstallations/blob/main/KeyFiguresElectricalInstallations36kV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kennzahlen Geobasisdatensatz Elektrische Anlagen mit einer Nennspannung von über 36kV

---

Notebook um Kennzahlen des Geobasisdatensatzes "Elektrische Anlagen mit einer Nennspannung von über 36kV abzuleiten.


**Offene Fragen**

*   Sollen bei Leitungen die Erdungen auch gezählt werden oder nicht?
*   Hat Swissgrid auch Stationen der Netzebene 4?

**ToDo**

*   Kennzahlen Stationen nach Netzebene 2 und 4 unterteilen





In [ ]:
#pip install geopandas

In [ ]:
#pip install datetime

In [43]:
import geopandas as gpd
from datetime import datetime

In [127]:
url = "https://data.geo.admin.ch/ch.bfe.elektrische-anlagen_ueber_36/gpkg/2056/ElektrischeAnlagenNennspannungUeber36kV_V1.gpkg"
dfLeitung = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Leitung')
dfStation = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Station')
dfMast = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Mast')

## Kennzahlen Leitungen

In [128]:
# Länge der Leitungen berechnen und nach Spannung aufsummieren
dfLeitung["Laenge"] = dfLeitung.length
dfLeitungKPI = dfLeitung.groupby(['StromnetzTyp','Spannung','LeitungTyp'])['Laenge'].sum()
dfLeitungKPI = dfLeitungKPI.to_frame().reset_index()
dfLeitungKPI['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [106]:
#Speichern
dfLeitungKPI.to_csv("Kennzahlen_Leitungen.csv", header=False, index=False, mode='a')

## Kennzahlen Masten

In [107]:
dfMastKPI = dfMast.groupby(['StromnetzTyp','MastTyp'])['xtf_id'].count().reset_index()
dfMastKPI = dfMastKPI.rename(columns={'xtf_id':'Anzahl'})
dfMastKPI['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [108]:
#Speichern
dfMastKPI.to_csv("Kennzahlen_Masten.csv", header=True, index=False, mode='a')

## Kennzahlen Stationen

In [111]:
dfStationKPI = dfStation.groupby(['StromnetzTyp','StationTyp'])['xtf_id'].count().reset_index()
dfStationKPI = dfStationKPI.rename(columns={'xtf_id':'Anzahl'})
dfStationKPI['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [113]:
#Speichern
dfStationKPI.to_csv("Kennzahlen_Stationen.csv", header=True, index=False, mode='a')


## Kennzahlen Werke

In [153]:
# Leitungslänge pro Werk ableiten.
dfWerkeLeitung = dfLeitung
dfWerkeLeitung['Werk'] = dfWerkeLeitung['xtf_id'].str.split('_').str[0]
dfWerkeLeitung = dfWerkeLeitung.groupby(['Werk'])['Laenge'].sum()
dfWerkeLeitung = dfWerkeLeitung.to_frame().reset_index()
dfWerkeLeitung = dfWerkeLeitung.rename(columns={'Laenge':'LaengeLeitung'})
dfWerkeLeitung

,Werk,LaengeLeitung
0,axpo,2.241945e+06
1,ibc,4.645698e+04
2,iwb,1.137061e+06
3,lagoule,2.334900e+04
4,swissgrid,1.058801e+07


In [154]:
# Anzahl Masten pro Werk ableiten.
dfWerkeMast = dfMast
dfWerkeMast['Werk'] = dfWerkeMast['xtf_id'].str.split('_').str[0]
dfWerkeMast = dfWerkeMast.groupby(['Werk'])['xtf_id'].count().reset_index()
dfWerkeMast = dfWerkeMast.rename(columns={'xtf_id':'AnzahlMasten'})

In [156]:
dfWerkeMast

,Werk,AnzahlMasten
0,axpo,7010
1,ibc,25
2,lagoule,339
3,swissgrid,11975


In [ ]:
dfWerkeMast = dfWerkeMast.groupby(['Werk'])['Laenge'].sum()
dfWerkeMast = dfWerkeMast.to_frame().reset_index()
dfWerkeMast


dfMastKPI = dfMast.groupby(['StromnetzTyp','MastTyp'])['xtf_id'].count().reset_index()



In [141]:
dfWerkeMast

,xtf_id,Mast_ID,Bezeichnung,Eigentuemer,EigentumVollstaendig,Lagegenauigkeit,StromnetzTyp,MastTyp,Hoehe,geometry,Werk
0,lagoule_A000000000000271,10276323717797171117912781,NaN,SEG,True,ungenau,Allgemeinstrom,Tragmast,NaN,POINT (2573863.169 1226958.713),lagoule
1,lagoule_A000000000000278,10276326023640180331606733,NaN,SEG,True,ungenau,Allgemeinstrom,Tragmast,NaN,POINT (2573899.087 1226966.450),lagoule
2,lagoule_A000000000000285,10276328329483189545300685,NaN,SEG,True,ungenau,Allgemeinstrom,Tragmast,NaN,POINT (2573827.292 1226950.986),lagoule
3,lagoule_A000000000000292,10276330635326198758994637,NaN,SEG,True,ungenau,Allgemeinstrom,Tragmast,NaN,POINT (2573791.414 1226943.258),lagoule
4,lagoule_A000000000000299,10276332941169207972688589,NaN,SEG,True,ungenau,Allgemeinstrom,Tragmast,NaN,POINT (2573755.537 1226935.532),lagoule
...,...,...,...,...,...,...,...,...,...,...,...
19344,axpo_253E5CF7-7E9A-4D46-B201-2EA34BCD4FAC,253E5CF7-7E9A-4D46-B201-2EA34BCD4FAC,TR0223 / Nr.:78,Axpo Grid AG,True,ungenau,Allgemeinstrom,Abspannmast,1.0,POINT (2702759.726 1227733.328),axpo
19345,axpo_85CFB19E-F42F-4EF6-A95C-293F60AAB913,85CFB19E-F42F-4EF6-A95C-293F60AAB913,TR0223 / Nr.:79,Axpo Grid AG,True,ungenau,Allgemeinstrom,Abspannmast,1.0,POINT (2702944.380 1227702.407),axpo
19346,axpo_3E3E0CC1-500A-40D7-BFA4-83F6A3B88998,3E3E0CC1-500A-40D7-BFA4-83F6A3B88998,TR0223 / Nr.:80,Axpo Grid AG,True,ungenau,Allgemeinstrom,Abspannmast,1.0,POINT (2703107.549 1227708.467),axpo
19347,axpo_18162611-F02A-428B-A6CF-A8D251CCC75F,18162611-F02A-428B-A6CF-A8D251CCC75F,TR0223 / Nr.:81,Axpo Grid AG,True,ungenau,Allgemeinstrom,Tragmast,1.0,POINT (2703257.632 1227731.102),axpo
